In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense




%matplotlib inline

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')
print(gpu_available)
print(tf.__version__)
# tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.16.2


In [3]:
# Initialising CNN
classifier = Sequential()

In [4]:
# Step 1 - Convolution
classifier.add(keras.Input(shape=(64, 64, 3)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))

2024-08-11 10:32:32.475057: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-08-11 10:32:32.475137: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-08-11 10:32:32.475153: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-08-11 10:32:32.475206: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-11 10:32:32.475237: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
loc = "data/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/Python/"

dataset = pd.read_csv(loc + 'Churn_Modelling.csv')
print(dataset.shape)
dataset.head()

In [ ]:
X = dataset.iloc[:, 3:12].values
y = dataset.iloc[:, 13].values

In [ ]:
X

In [ ]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("Geography, Gender", OneHotEncoder(categories='auto'),[1, 2])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
X = ct.fit_transform(X)  
X = X[:, 1:]
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [ ]:
# Feature scaling
sc_X = StandardScaler(with_mean=False)
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Initialising the ANN
classifier = Sequential()

In [ ]:
# Adding the inpput layer and the first hidden layer
classifier.add(keras.Input(shape=(11,)))
classifier.add(Dense(units=6, kernel_initializer="glorot_uniform", activation='relu'))
classifier.add(Dense(units=6, kernel_initializer="glorot_uniform", activation='relu'))
classifier.add(Dense(units=1, kernel_initializer="glorot_uniform", activation='sigmoid'))

In [ ]:
# Compiling th ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

In [ ]:
# Fitting the ANN to the Trainning set
# classifier.fit(X_train, y_train, batch_size=10, epochs=100)

In [ ]:
model_file = 'models/ann_classifier.keras'
try:
  # Specify an invalid GPU device
  with tf.device('/GPU:0'):
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                    patience = 3, restore_best_weights = False)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_file, 
                        monitor="loss", mode="min", 
                        save_best_only=True, verbose=1)
    history = classifier.fit(X_train, y_train, batch_size=10, epochs=100, callbacks=[early_stop, checkpoint])
except RuntimeError as e:
    print(e)
# loading model weights
# classifier.load_weights(model_file)

In [ ]:
# try:
#   # Specify an invalid GPU device
#   with tf.device('/GPU:0'):
#     history = classifier.fit(X_train, y_train, batch_size=10, epochs=100)
# except RuntimeError as e:
#   print(e)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > .5)
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# array([[1533,   62],
#        [ 255,  150]])

In [ ]:
# array([[1533,   62],
#        [ 210,  195]])

In [ ]:
 # array([[1539,   56],
 #       [ 266,  139]])

In [ ]:
# array([[1542,   53],
#        [ 274,  131]])

In [ ]:
# array([[1550,   45],
#        [ 230,  175]])

In [ ]:
# array([[1505,   90],
#        [ 184,  221]])

In [ ]:
# array([[1533,   62],
#        [ 218,  187]])